## ДЛЯ ПОДСЧЕТА выгрузок - 011123

In [172]:
import pandas as pd
import numpy as np
from IPython.display import display_markdown
import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-danger">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b>функция<br>
</b>            </div>

In [173]:
#     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла 605го ----------------------


def def_zagruz(file):
    # file_name = '1c_605_0111_1400.csv'
    file_name = file
    # file_name = 'выгрузка_из_1с_0111_1800.csv'
    df = pd.read_csv(file_name, delimiter=';', encoding='cp1251')

    df_605 = df.copy()
    df_605.shape

    # уберем лишние колонки
    df_605 = df.copy()
    df_605 = df_605[[
        'Дата постановки на учет/включения пациента в реестр',
     'Пациент',
     'Пол',
     'Дата рождения',
     'Гражданский статус',
     'Адрес',
     # 'Адрес фактический',
     'Документ удостоверяющий личность',
     'Серия/№',
     # 'Кем выдан',
     # 'Свидетельство о регистрации по месту жительства(до 14 лет)',
     'СНИЛС',
     'Полис ОМС',
     # 'Наличие инвалидности',
     'Основной диагноз по МКБ-10',
     # 'Основное контактное лицо',
     'Дата признания пациента нуждающимся в ПМП (ВК)',
     'Тип документа о признании паллиативным',
     'Медицинская организация, в которой признали пациента нуждающимся в ПМП',
     'Наличие разрешения ДЗМ на оказание помощи',
     'Дата разрешения ДЗМ',
     'Номер разрешения ДЗМ',
     'Текущий статус пациента',
     'Дата прекращения оказания ПМП',
     'Основания прекращения оказания пациенту ПМП', #------------------
     'Курирующее подразделение/филиал',
     'Наблюдение в РЦ',
     'БК',
     'Основной диагноз на момент обращения',
     'Комментарий',
     'Участник СВО', #------------------
     'Канал поступления информации',  #------------------
     'Кто направил',  #------------------
     'Дата получения ИДС на передачу данных',  #------------------
     '№ амбулаторной карты',
        'Дата включения в реестр пациентов, нуждающихся в ПМП'
                         ]]

    #  переименуем покороче столбцы
    df_605.rename(columns={'Дата постановки на учет/включения пациента в реестр': 'Дата_постанов'}, inplace=True)
    df_605.rename(columns={'Дата рождения': 'ДР'}, inplace=True)
    df_605.rename(columns={'Дата признания пациента нуждающимся в ПМП (ВК)': 'Дата_признания'}, inplace=True)
    df_605.rename(columns={'Дата прекращения оказания ПМП': 'Дата_прекр'}, inplace=True)
    df_605.rename(columns={'Курирующее подразделение/филиал': 'Филиал'}, inplace=True)
    df_605.rename(columns={'Основной диагноз по МКБ-10': 'Диагноз'}, inplace=True)
    df_605.rename(columns={'Гражданский статус': 'Иногород'}, inplace=True)
    df_concat = pd.concat([df_605])
    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_постанов'] = pd.to_datetime(df_concat['Дата_постанов'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()

    # Уберем записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ' --------------- БК
    # - удаляем строки. в которых текущий статус не заполнен =  nan

  
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isin(['Новый пациент', 'Умер без п/о', 'Вне реестра', 'Врач КЦ']))]   # -----------------записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'----------------
    # df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isna())]  # - удаляем строки. в которых текущий статус не заполнен =  nan

    df_concat = df_concat.loc[(df_concat['БК'] == 'Нет')]  # - удаляем строки. в которых текущий статус не заполнен =  nan
    df_concat = df_concat.loc[(~df_concat['Филиал'].isin(['ГБУЗ Московский многопрофильный центр паллиативной','Координационный центр','БМЧУ "Детский хоспис"']) )]  # - удаляем строки. в которых текущий статус не заполнен =  nan


    
    # -------- расчет разницы 
    df_concat['diff_days'] =(df_concat['Дата_прекр'] - df_concat['Дата_постанов']) / np.timedelta64 ( 1 , 'D')

    
    df_concat['Тип_диагноза'] = np.nan

    return df_concat

In [175]:
def translete_diagnoz_startswith(df, list_diagn, new_type_diagnog):
    for list_diagn_ in list_diagn:
        df['Диагноз'] = df['Диагноз'].str.lower()
        df.loc[  
               # (df['Тип_диагноза'].isna())
            (~df['Диагноз'].isna())
                 & 
            (df['Диагноз'].str.startswith(list_diagn_)), 'Тип_диагноза' ] = new_type_diagnog

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b>Считаем ДИАГНОЗЫ<br>
</b>            </div>

In [178]:
def translete_diagnoz_startswith(df, list_diagn, new_type_diagnog):
    for list_diagn_ in list_diagn:
        df['Диагноз'] = df['Диагноз'].str.lower()
        df.loc[  
               # (df['Тип_диагноза'].isna())
            (~df['Диагноз'].isna())
                 & 
            (df['Диагноз'].str.startswith(list_diagn_)), 'Тип_диагноза' ] = new_type_diagnog

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b>Описание признаков словосочетаний списков ДИАГНОЗОВ<br>
</b>            </div>

In [179]:
list_startswith_onko = [    'c',    '(c',    'с', '(d', 'd',
                       'злокачественная меланома туловища',
                        'злокачественное новообразование селезеночного изгиба', 
                       'доброкачественное новообразование поджелудочной железы',
                       'глиобластома головного мозга', 
                       ]
list_startswith_vich = [    'в20' , 'b20', 'вич', ]
list_startswith_infection = [  'b',  ]
 
list_startswith_teraphiya = [  'd5', 'd6', 'd7', 'd8',
                                'd 5', 'd 6', 'd 7', 'd 8' ,
                                'e', '(e', 'е0',
                                 'h', '(h',
                                  'i', '(i',
                                  'j', '(j',
                                  'k', '(k', 'к',
                                  'l', '(l',
                                  'm', 
                                  'n',
                                  'm',     '(m',   '(м','м'                   
                                 's',      
                             'другие уточненные поражения сосудов мозга',
       'ревматоидный артрит', 
       ' алкогольный цирроз печени', 
                             'атеросклеротическая болезнь сердца',
                             'цвб хигм',
       # 'диагнозтест', 
                             'ушиб головного мозга',
       
                             'вич-инфекция',
       'хроническая обструктивн'
                                                   ]
                             
list_startswith_psih = [  'f','(f',]
list_startswith_nevrol = [  'g', '(g',]              
list_startswith_vrojden = [  'q', '(q',]    
list_startswith_proch = [  
                         '(a',
                        'r', 
                         's','(s', 
                         'u', '(u',
                         'v',
                         'z',
                         '(r',
                         '(p', 'p',
                         't', 'y', 'т']         

In [180]:
# df_result.columns.tolist()

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=5>
<b> РЕЗУЛЬТАТ<br>
</b>            </div>

In [181]:
# file_name = 'выгрузка_из_1с_0111_1800.csv'
# file_name = 'Выгрузка_из_1с_0211.csv'
# file_name = '1c_0211_1640.csv'
# file_name = 'выгрузка_из_1с_0311.csv'
# file_name ='Реестр 605.csv'


# file_name = 'Реестр 03111835 для перепроверки.csv'
# file_name = '1с_0711.csv'

file_name = '1с_повключению_все_0711.csv'



df_result = def_zagruz(file_name)

# --------------- диагн ---------
df_result['Тип_диагноза'] = np.nan

# ------------- применим функции
translete_diagnoz_startswith(df_result, list_startswith_onko, 'онко')
translete_diagnoz_startswith(df_result, list_startswith_vich, 'vich')
translete_diagnoz_startswith(df_result, list_startswith_infection, 'infek')
translete_diagnoz_startswith(df_result, list_startswith_teraphiya, 'teraph')
translete_diagnoz_startswith(df_result, list_startswith_psih, 'psih')
translete_diagnoz_startswith(df_result, list_startswith_nevrol, 'nevrol')
translete_diagnoz_startswith(df_result, list_startswith_vrojden, 'vrojden')
translete_diagnoz_startswith(df_result, list_startswith_proch, 'proch')
# -----------------------------------------
df_result.loc[df_result['Тип_диагноза'].isna(), 'Тип_диагноза'] = 'not'
# 
# df_result

In [182]:
df_result.columns.tolist()

['Дата_постанов',
 'Пациент',
 'Пол',
 'ДР',
 'Иногород',
 'Адрес',
 'Документ удостоверяющий личность',
 'Серия/№',
 'СНИЛС',
 'Полис ОМС',
 'Диагноз',
 'Дата_признания',
 'Тип документа о признании паллиативным',
 'Медицинская организация, в которой признали пациента нуждающимся в ПМП',
 'Наличие разрешения ДЗМ на оказание помощи',
 'Дата разрешения ДЗМ',
 'Номер разрешения ДЗМ',
 'Текущий статус пациента',
 'Дата_прекр',
 'Основания прекращения оказания пациенту ПМП',
 'Филиал',
 'Наблюдение в РЦ',
 'БК',
 'Основной диагноз на момент обращения',
 'Комментарий',
 'Участник СВО',
 'Канал поступления информации',
 'Кто направил',
 'Дата получения ИДС на передачу данных',
 '№ амбулаторной карты',
 'Дата включения в реестр пациентов, нуждающихся в ПМП',
 'diff_days',
 'Тип_диагноза']

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> применим функции для диагнозов<br>
</b>            </div>

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Диагноззы <br>
</b>            </div>

In [183]:
# ---------------------------------------------- диагноз - ПУСТО
diag_pusto = df_result.loc[df_result['Тип_диагноза'] == 'not']
# ---------------------------------------------- диагноз - ПУСТО на патронаже -----
diag_pusto_patron = df_result.loc[(df_result['Тип_диагноза'] == 'not') & (df_result['Текущий статус пациента'] == 'Патронаж')]
# ---------------------------------------------- диагноз - неконкрет
diag_konkret = df_result.loc[             (df_result['Диагноз'].str.len() < 9)                           ]
# ---------------------------------------------- диагноз - конкрет
diag_ne_konkret = df_result.loc[             (df_result['Диагноз'].str.len()       > 10                 ) ]



<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> 0 - 3 <br>
</b>            </div>

In [184]:
# #------------------------------ --------------------- 0 - 3 -------------
# df_result.loc[ 
#        (df_result['diff_days'] >= 0) 
#     &  (df_result['diff_days'] <= 3) 
#      & (df_result['Тип_диагноза'] == 'онко')
#     & (df_result['Дата_постанов'] >= '2023-01-01')
# ]

In [91]:
# #------------------------------ --------------------- 0 - 3 -------------
# df_result.loc[ 
#        (df_result['diff_days'] >= 0) 
#     &  (df_result['diff_days'] <= 3) 
#      & (df_result['Тип_диагноза'] == 'онко')
#     & (df_result['Дата_постанов'] >= '2023-01-01')
#     & (df_result['Текущий статус пациента'] == 'Прекращено оказание ПМП')
#              ]

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Адреса <br>
</b>            </div>

In [92]:
# ---------------------------------------------- адрес - пустой
adres_pusto = df_result.loc[df_result['Адрес'].isna()]

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Иногород <br>
</b>            </div>

In [93]:
# ---------------------------------------------- Иногород статус - пустой
inogorod_pusto = df_result.loc[
    # (df_result['Иногород'] != 'Москвич')
   (df_result['Иногород'].isna())
]

In [94]:
# ---------------------------------------------- Иногород статус - немоскв заполненнеы --- 
inogorod_vse = df_result.loc[
    (df_result['Иногород'] != 'Москвич')
   & (~df_result['Иногород'].isna())
]
# inogorod_vse

In [95]:
# ---------------------------------------------- Иногород статус - пустой  разреш
inogorod_s_razresh_22 = df_result.loc[
    (df_result['Иногород'] != 'Москвич')
   & (~df_result['Иногород'].isna())
   & (
       (df_result['Наличие разрешения ДЗМ на оказание помощи'] == 'Да')
        & ((df_result['Дата_постанов'] >='2022-01-01' ) & (df_result['Дата_постанов'] <='2023-01-01' ))
            ) 
]
inogorod_s_razresh_22.shape

(81, 32)

In [96]:
# ---------------------------------------------- Иногород статус -  c разреш 2023
inogorod_s_razresh_23 = df_result.loc[
    (df_result['Иногород'] != 'Москвич')
   & (~df_result['Иногород'].isna())
   & (
       (df_result['Наличие разрешения ДЗМ на оказание помощи'] == 'Да')
        & ((df_result['Дата_постанов'] >='2023-01-01' ) )
            ) 
]
inogorod_s_razresh_23.shape

(159, 32)

In [97]:
# ----------------------------------------------  Иногород статус - без разреш 2023
inogorod_s_razresh_23 = df_result.loc[
    (df_result['Иногород'] != 'Москвич')
   & (~df_result['Иногород'].isna())
   & (
       (df_result['Наличие разрешения ДЗМ на оказание помощи'] == 'Нет')
        & ((df_result['Дата_постанов'] >='2023-01-01' ) )
            ) 
]
inogorod_s_razresh_23.shape

(6, 32)

In [98]:
inogorod_s_razresh_23

,Дата_постанов,Пациент,Пол,ДР,Иногород,Адрес,Документ удостоверяющий личность,Серия/№,СНИЛС,Полис ОМС,...,БК,Основной диагноз на момент обращения,Комментарий,Участник СВО,Канал поступления информации,Кто направил,Дата получения ИДС на передачу данных,№ амбулаторной карты,diff_days,Тип_диагноза
99025,2023-05-19,Елина Нина Алексеевна,Женский,1940-12-12,Гражданин РФ/иногородний,"Тверская обл, г.о. город Тверь, Урицкого, дом ...",Паспорт,2800 259961,013-291-712 14,6967 9508 8700 0119,...,Нет,Опухоль ободочной кишки с поражением регионарн...,31.10.2023 - первичный визит на 19.05.2023 был...,NaN,Самообращение,родственники,NaN,14/468/23,2.0,онко
102874,2023-08-01,Вартанов Гайк Грантович,Мужской,1948-01-10,Гражданин РФ/иногородний,"Московская обл, Сергиев Посад г, Октябрьская у...",Паспорт,4604604292,019-762-456 78,7778 1508 3900 1847,...,Нет,Ишемия сосудов головного мозга,Данные получены в телефонном режиме. \nВрачу ...,NaN,Почта,ГП 22. ф.1,NaN,16/807/23,NaN,teraph
106225,2023-10-04,Барышников Михаил Иванович,Мужской,1945-05-30,Гражданин РФ/иногородний,"Москва г, вн.тер.г. муниципальный округ Нагорн...",Паспорт,4523 579903,004-610-335 89,7774 4508 1900 0080,...,Нет,J44.8 - Другая уточненная хроническая обструкт...,Данные получены в телефонном режиме. Врачу пр...,NaN,Почта,родственники,NaN,16/1019/23,NaN,teraph
107794,2023-11-03,Касымов Фарид Мухамедович,Мужской,1967-05-16,Гражданин РФ/иногородний,"109125, Москва г, муниципальный округ Рязански...",Паспорт,4512 753217,060-618-737 54,7754 2308 3300 2336,...,Нет,C10.8 - Рак ротоглотки,Пациент постоянно прописан в МО. Сестра сказал...,нет,Почта,НМЦ им. А.С. Логинова,03.11.2023,Р/470/23,NaN,онко
107874,2023-11-04,Филатова Лариса Николаевна,Женский,1976-07-11,БОМЖ,NaN,Паспорт,4621/258630,025-741-094 38,7752 3208 8800 1473,...,Нет,С50.9,"НОМЕР УКАЗАН НЕ ВЕРНО, ЗАПРОШЕНЫ КОНТАКТЫ!!!\n...",нет,Почта,ГП №22 ф.2,NaN,16/1136/23,NaN,not
107978,2202-04-12,Еникова Феня Мартковна,Женский,1925-11-18,Гражданин РФ/иногородний,Крым Феодосия Украинская 11,NaN,4514652788,18538624608,7768 4708 8100 0873,...,Нет,Деменция,NaN,NaN,NaN,NaN,NaN,1/339/21,-66100.0,teraph


In [99]:
inogorod_s_razresh.columns.tolist()

['Дата_постанов',
 'Пациент',
 'Пол',
 'ДР',
 'Иногород',
 'Адрес',
 'Документ удостоверяющий личность',
 'Серия/№',
 'СНИЛС',
 'Полис ОМС',
 'Диагноз',
 'Дата_признания',
 'Тип документа о признании паллиативным',
 'Медицинская организация, в которой признали пациента нуждающимся в ПМП',
 'Наличие разрешения ДЗМ на оказание помощи',
 'Дата разрешения ДЗМ',
 'Номер разрешения ДЗМ',
 'Текущий статус пациента',
 'Дата_прекр',
 'Филиал',
 'БК',
 'diff_days',
 'Тип_диагноза']

In [100]:
inogorod_s_razresh[[

'Дата_постанов',
 'Пациент',
 'Пол',
 'ДР',
 'Иногород',
 'Адрес',
 'Документ удостоверяющий личность',
 'Серия/№',
 'СНИЛС',
 'Полис ОМС',
 'Диагноз',
 'Дата_признания',
 'Тип документа о признании паллиативным',
 'Медицинская организация, в которой признали пациента нуждающимся в ПМП',
 'Наличие разрешения ДЗМ на оказание помощи',
 'Дата разрешения ДЗМ',
 'Номер разрешения ДЗМ',
 'Текущий статус пациента',
 'Дата_прекр',
 'Основания прекращения оказания пациенту ПМП',
 'Филиал',
 'Наблюдение в РЦ',
 'БК',

]]

KeyError: "['Основания прекращения оказания пациенту ПМП', 'Наблюдение в РЦ'] not in index"

In [101]:
# ---------------------------------------------- Иногород статус - пустой  БЕЗ разреш
inogorod_bez_razresh = df_result.loc[
    (df_result['Иногород'] != 'Москвич')
   & (~df_result['Иногород'].isna())
   & (
   (df_result['Наличие разрешения ДЗМ на оказание помощи'] == 'Нет')
    | (df_result['Дата разрешения ДЗМ'].isna())
    | (df_result['Номер разрешения ДЗМ'].isna())
    ) 
]
# inogorod_bez_razresh

In [102]:
# ----------------------------------------------  Иногород  без адреса
inogorod_bez_adres = df_result.loc[
    (df_result['Иногород'] != 'Москвич')
   & (~df_result['Иногород'].isna())
    & (df_result['Адрес'].isna())
]
# inogorod_bez_adres

In [103]:
# ---------------------------------------------- Иногород статус - пустой  разреш
inogorod_s_razresh = df_result.loc[
    (df_result['Иногород'] != 'Москвич')
   & (~df_result['Иногород'].isna())
   & (
       (df_result['Наличие разрешения ДЗМ на оказание помощи'] == 'Да')
        | (~df_result['Дата разрешения ДЗМ'].isna())
        | (~df_result['Номер разрешения ДЗМ'].isna())
       & (df_result['Номер разрешения ДЗМ'])    
    ) 
]
inogorod_s_razresh.shape

(1093, 32)

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Прекращено ПМП <br>
</b>            </div>

In [104]:
# ---------------------------------------------- Прекращено ПМП
end_pmp_vse = df_result.loc[(df_result['Текущий статус пациента'] == 'Прекращено оказание ПМП')]
end_pmp_vse.shape

(1901, 32)

In [105]:
# ---------------------------------------------- Прекращено ПМП  --- записей без даты прекращения
end_pmp_bez_date_end = df_result.loc[
    (df_result['Текущий статус пациента'] == 'Прекращено оказание ПМП')
   & (df_result['Дата_прекр'].isna())
]
end_pmp_bez_date_end.shape

(424, 32)

In [106]:
# ---------------------------------------------- Прекращено ПМП  --- записей с кривой датой прекращения
end_pmp_kriv_date = df_result.loc[
    (df_result['Текущий статус пациента'] == 'Прекращено оказание ПМП')
     & (~df_result['Дата_прекр'].isna())
   & ((df_result['Дата_прекр'] >= pd.to_datetime('today').normalize()) | (df_result['Дата_прекр'] <= '1900-01-01') )
]
end_pmp_kriv_date.shape

(1, 32)

In [108]:
# ----------------------------------------------  Прекращено ПМП записей без обоснования (поле обоснования пусто) - 1181
end_pmp_bez_obosnovan =  df_result.loc[(df_result['Текущий статус пациента'] == 'Прекращено оказание ПМП')
                                         & (df_result['Основания прекращения оказания пациенту ПМП'].isna())]
end_pmp_bez_obosnovan.shape

(1053, 32)

In [109]:
# ----------------------------2023 год------------------ Прекращено ПМП 2023 всего записей в 2023 г (постановка + выбытие)  ** 1500
end_pmp_vse_2023 =  df_result.loc[
                                     (df_result['Текущий статус пациента'] == 'Прекращено оказание ПМП')
                                    & ((df_result['Дата_прекр'] >= '2023-01-01') & (df_result['Дата_прекр'] <= '2023-11-01'))
                                       # &(df_result['Тип_диагноза'] == 'онко')
                                       ]
# end_pmp_vse_2023

In [110]:
# ----------------------------2023 год------------------ Прекращено ПМП 2023 всего записей в 2023 г  без обоснования (поле обоснования пусто)
end_pmp_2023_bez_obosnovan =  df_result.loc[
                                     (df_result['Текущий статус пациента'] == 'Прекращено оказание ПМП')
                                    & ((df_result['Дата_прекр'] >= '2023-01-01') & (df_result['Дата_прекр'] <= '2023-11-01'))
                                    & (df_result['Основания прекращения оказания пациенту ПМП'].isna())
                                       ]
# end_pmp_2023_bez_obosnovan

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Дата постановки <br>
</b>            </div>

In [111]:
# ----------------------Дата постановки на учет	количество незаполненных - 2103
date_postanov = df_result.loc[
   (df_result['Дата_постанов'].isna())
                          ]
# date_postanov.shape

In [112]:
# ----------------------Дата постановки на учет	количество незаполненных - 2103
date_postanov_kriv = df_result.loc[
                                   (~df_result['Дата_постанов'].isna())
                                 & ((df_result['Дата_постанов'] >= pd.to_datetime('today').normalize()) | (df_result['Дата_постанов'] <= '2010-01-01') )
                                                              ]
# date_postanov_kriv.shape

In [113]:
# ----------------------Дата признания	количество незаполненных количество незаполненных в 2023 (по выбытию и живые) -  19098
date_postanov_pusto_2023 =  df_result.loc[
                                     (df_result['Дата_постанов'].isna())
                                   & ( (df_result['Дата_прекр'] >= '2023-01-01') | (df_result['Дата_прекр'].isna())) 
                                           ]
date_postanov_pusto_2023.shape

(453, 32)

In [114]:
# date_postanov_pusto_2023

In [115]:
date_postanov_pusto_2023.to_excel('sss.xlsx')

In [116]:
# ----------------------Дата признания	количество незаполненных количество незаполненных в 2023 (по выбытию и живые) -  19098
date_postanov_kriv_2023 =  df_result.loc[
                                     (~df_result['Дата_постанов'].isna())
                                   & ((df_result['Дата_прекр'] >= '2023-01-01') & (df_result['Дата_прекр'] <= '2023-11-01') | (df_result['Дата_прекр'].isna())) 
                                   & ((df_result['Дата_постанов'] >= pd.to_datetime('today').normalize()) | (df_result['Дата_постанов'] <= '2010-01-01') )
                                           ]
# date_postanov_kriv_2023.shape

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Дата признания <br>
</b>            </div>

In [117]:
# ----------------------Дата признания	количество незаполненных  -59097  
date_priznan_pusto = df_result.loc[
                             (df_result['Дата_признания'].isna())
                                  ]
# date_priznan_pusto.shape

In [118]:
# ----------------------Дата признания	количество кривых записей (включ 2010 и раньше, будущее)
date_priznan_kriv = df_result.loc[
                             (~df_result['Дата_признания'].isna())
                          & ((df_result['Дата_постанов'] >= pd.to_datetime('today').normalize()) | (df_result['Дата_постанов'] <= '2010-01-01') )
                                      ]
date_priznan_kriv.shape

(21, 32)

In [119]:
# ----------------------Дата признания	количество незаполненных в 2023 (по выбытию и живые)
date_priznan_pusto_2023 = df_result.loc[
                             (df_result['Дата_признания'].isna())
                         & ((df_result['Дата_прекр'] >= '2023-01-01') & (df_result['Дата_прекр'] <= '2023-11-01') | (df_result['Дата_прекр'].isna())) 
                       # & (df_result['Тип_диагноза'] == 'онко') # -------------------- diag
                                      ]
date_priznan_pusto_2023.shape

(18186, 32)

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Медорганиз <br>
</b>            </div>

In [120]:
# df_result

In [121]:
def new_fich_medorg_kriv(df, list_kriv_med):
    for list_kriv_med_ in list_kriv_med:
        # df['Диагноз'] = df['Диагноз'].str.lower()
        df.loc[  
               
            (~df['Медицинская организация, в которой признали пациента нуждающимся в ПМП'].isna())
                 & 
            (df['Медицинская организация, в которой признали пациента нуждающимся в ПМП'].str.startswith(list_kriv_med_)), 'Признак_неправильной_МО' ] = 'Некорректная_МО'
    return df

In [122]:
# -------------------------------------Медорганизация, призн ПМП	количество незаполненных
medorg_pusto = df_result.loc[
                         (df_result['Медицинская организация, в которой признали пациента нуждающимся в ПМП'].isna())

                                          ]
medorg_pusto.shape

(6670, 32)

In [123]:
# -------------------------------------Медорганизация, призн ПМП	количество незаполненных в 2023 (по выбытию и живые) !!!!!!!!!!!!!!!!!!!!!!!!!!!!
medorg_pusto_2023 = df_result.loc[
                         (df_result['Медицинская организация, в которой признали пациента нуждающимся в ПМП'].isna())
                         & (
                               ((df_result['Дата_прекр'] >= '2023-01-01') | (df_result['Дата_прекр'].isna()))
                           )
                                          ]
medorg_pusto_2023.shape

(1617, 32)

In [124]:
# df_result

In [125]:
# -------------------------------------Медорганизация, КРИВЫЕ !!!!!!!!!!!!!!!!!!!!!!!!!!!!
list_medorg_kriv = [
'-',
'нет',
'2021-04-30',
'20',
'гистология',
'6 ОПМП',
 'ВК',   
]

df_result['Признак_неправильной_МО'] = np.nan
df_result = new_fich_medorg_kriv(df_result, list_medorg_kriv)

medorg_vse_kriv = df_result.loc[ (~df_result['Признак_неправильной_МО'].isna())
                                          ]
medorg_vse_kriv.shape

(50452, 33)

In [126]:
medorg_vse_kriv['Медицинская организация, в которой признали пациента нуждающимся в ПМП'].unique().tolist()

['-',
 'нет                                                ГБУЗ МО "Щелковская областная больница"',
 'нет                                                              ГБУЗ ГП №107 ДЗМ',
 'нет нет',
 'ВК',
 'гистология',
 'нет данных',
 'нет                                                   ГБУЗ ГП №218 ДЗМ',
 'нет ГБУЗ ГКОБ №1 ДЗМ']

<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=1>
<b> Дубли <br>
</b>            </div>

In [127]:
 # df_result[df_result.duplicated(['Пациент', 'ДР'], keep=False)].sort_values('Пациент').loc[~df_result['ДР'].isna()]

In [128]:
# -------------------Дубли ФИО ДР
dubl_fio_dr = df_result[  df_result.duplicated(['Пациент', 'ДР'], keep=False  )]
dubl_fio_dr.shape

(14, 33)

In [129]:
# -------------------Дубли Полис ОМС
dubl_oms = df_result[  df_result.duplicated(['Полис ОМС'], keep=False)  ].loc[
                                                       (~df_result['Полис ОМС'].isna())]
dubl_oms.shape

(473, 33)

In [130]:
# -------------------Дубли СНИЛС
dubl_snils = df_result[  df_result.duplicated(['СНИЛС'], keep=False)  ].loc[
                                                       (~df_result['СНИЛС'].isna())
]
dubl_snils.shape

(518, 33)

In [131]:
# -------------------Дубли ДУЛ
dubl_dul = df_result[  df_result.duplicated(['Серия/№'], keep=False)  ].loc[
                                                       (~df_result['Серия/№'].isna())
                    &  (df_result['Серия/№'] != '0')
                    &  (df_result['Серия/№'] != '0000000')
                    &  (df_result['Серия/№'] != '000000')
    &  (df_result['Серия/№'] != '00000')
    &  (df_result['Серия/№'] != '00000000')
    &  (df_result['Серия/№'] != '00000000000')
    &  (df_result['Серия/№'] != '0000')
    &  (df_result['Серия/№'] != '0000000000')  
        
]
dubl_dul.shape

(121, 33)

In [132]:
df_result.loc[df_result['Тип_диагноза'] == 'not']

a = f'Статистика реестра (выгрузка) "{file_name}":'

display_markdown(f''' # {a}

| Параметр | Количество | прим |
|---:|:-|:-|
| __Всего В РАССМОТРЕНИИ__      | {df_result.shape[0]}      | - |
| Количество пациентов, с незаполненным __диагнозом__          | {diag_pusto.shape[0]}   | на патронаже  {diag_pusto_patron.shape[0]}    |
| Количество пациентов, "неконкретным" диагнозом               | {diag_ne_konkret.shape[0]}   |    |
| Количество пациентов, "конкретным" диагнозом               | {diag_konkret.shape[0]}   |    |
| -- адрес - пустой           | {adres_pusto.shape[0]}   |    |
| __Гражд статус__ - пустой            | {inogorod_pusto.shape[0]}   |    |
| -- Иногород all         | {inogorod_vse .shape[0]}   |    |
| -- Иногород статус - c разреш         | {inogorod_s_razresh.shape[0]}   |    |
| -- Иногород статус -  БЕЗ разреш         | {inogorod_bez_razresh .shape[0]}   |    |
| -- inogorod_bez_adres          | {inogorod_bez_adres .shape[0]}   |    |
| __Прекращено ПМП__ end_pmp_vse           | {end_pmp_vse .shape[0]}   |    |
| end_pmp_bez_date_end           | {end_pmp_bez_date_end .shape[0]}   |    |
| --end_pmp_kriv_date           | {end_pmp_kriv_date .shape[0]}   |    |
| --end_pmp_bez_obosnovan           | {end_pmp_bez_obosnovan .shape[0]}   |    |
| -- end_pmp_vse_2023          | {end_pmp_vse_2023 .shape[0]}   |    |
| --  end_pmp_2023_bez_obosnovan         | {end_pmp_2023_bez_obosnovan.shape[0]}   |    |
| __Дата постановки__ date_postanov пусто          | {date_postanov .shape[0]}   |    |
| -- date_postanov_kriv          | {date_postanov_kriv .shape[0]}   |    |
| --  date_postanov_pusto_2023         | {date_postanov_pusto_2023 .shape[0]}   |    |
| --  date_postanov_kriv_2023         | {date_postanov_kriv_2023 .shape[0]}   |    |
|__Дата признания__  date_priznan_pusto          | {date_priznan_pusto .shape[0]}   |    |
| --date_priznan_kriv           | {date_priznan_kriv .shape[0]}   |    |
| --  date_priznan_pusto_2023         | {date_priznan_pusto_2023 .shape[0]}   |    |
| __Медорганиз__    medorg_pusto       | {medorg_pusto .shape[0]}   |    |
| -- medorg_pusto_2023          | {medorg_pusto_2023 .shape[0]}   |    |
| -- medorg_vse_kriv          | {medorg_vse_kriv .shape[0]}   |    |
| __Дубли__ dubl_fio_dr          | {dubl_fio_dr .shape[0]}   |    |
| --   dubl_oms        | {dubl_oms .shape[0]}   |    |
| --   dubl_snils        | {dubl_snils .shape[0]}   |    |
| --  dubl_dul         | {dubl_dul .shape[0]}   |    |


''', raw=True)

 # Статистика реестра (выгрузка) "1с_0711.csv":

| Параметр | Количество | прим |
|---:|:-|:-|
| __Всего В РАССМОТРЕНИИ__      | 97993      | - |
| Количество пациентов, с незаполненным __диагнозом__          | 1512   | на патронаже  315    |
| Количество пациентов, "неконкретным" диагнозом               | 83806   |    |
| Количество пациентов, "конкретным" диагнозом               | 13756   |    |
| -- адрес - пустой           | 3149   |    |
| __Гражд статус__ - пустой            | 2277   |    |
| -- Иногород all         | 1980   |    |
| -- Иногород статус - c разреш         | 1093   |    |
| -- Иногород статус -  БЕЗ разреш         | 1214   |    |
| -- inogorod_bez_adres          | 473   |    |
| __Прекращено ПМП__ end_pmp_vse           | 1901   |    |
| end_pmp_bez_date_end           | 424   |    |
| --end_pmp_kriv_date           | 1   |    |
| --end_pmp_bez_obosnovan           | 1053   |    |
| -- end_pmp_vse_2023          | 221   |    |
| --  end_pmp_2023_bez_obosnovan         | 2   |    |
| __Дата постановки__ date_postanov пусто          | 1996   |    |
| -- date_postanov_kriv          | 87   |    |
| --  date_postanov_pusto_2023         | 453   |    |
| --  date_postanov_kriv_2023         | 22   |    |
|__Дата признания__  date_priznan_pusto          | 58156   |    |
| --date_priznan_kriv           | 21   |    |
| --  date_priznan_pusto_2023         | 18186   |    |
| __Медорганиз__    medorg_pusto       | 6670   |    |
| -- medorg_pusto_2023          | 1617   |    |
| -- medorg_vse_kriv          | 50452   |    |
| __Дубли__ dubl_fio_dr          | 14   |    |
| --   dubl_oms        | 473   |    |
| --   dubl_snils        | 518   |    |
| --  dubl_dul         | 121   |    |




In [168]:
# **************
df_result.groupby(['Тип_диагноза']).aggregate({'Тип_диагноза':'count', 'Пациент':'count'})

,Тип_диагноза,Пациент
Тип_диагноза,,
infek,19,19
nevrol,6945,6945
not,1507,1507
proch,684,684
psih,108,108
teraph,21144,21144
vich,3,3
vrojden,95,95
онко,67502,67502


In [162]:
df_result_star_vkl_all = df_result.copy()
df_result_star_vkl_all

,Дата_постанов,Пациент,Пол,ДР,Иногород,Адрес,Документ удостоверяющий личность,Серия/№,СНИЛС,Полис ОМС,...,Основной диагноз на момент обращения,Комментарий,Участник СВО,Канал поступления информации,Кто направил,Дата получения ИДС на передачу данных,№ амбулаторной карты,"Дата включения в реестр пациентов, нуждающихся в ПМП",diff_days,Тип_диагноза
0,2023-02-02,Зинатшина Анастасия Владимировна,Женский,1986-09-22,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",Паспорт,4519056415,201-154-322 87,7750 3108 7700 4903,...,Рак желулка,"Со слов мужа Не ест ( тошнота),большая потеря ...",NaN,Почта,родственники,02.02.2023,6/49/23,01.02.2023,0.0,онко
1,2023-06-02,Олейникова Александра Павловна,Женский,1985-03-04,Москвич,"109341, Москва г, муниципальный округ Марьино,...",Паспорт,4520/883847,143-810-402 28,7756 4108 9500 0646,...,C18.0 - Злокачественное новообразование слепой...,"боли, НПВС без эффекта, поднимается иногда тем...",NaN,Почта,родственники,02.06.2023,6/248/23,30.05.2023,26.0,онко
3,2020-05-15,Волченкова Юлия Викторовна,Женский,1983-05-22,Москвич,"Москва г, Зеленоград г, корпус 810, квартира 110",Паспорт,4614 693375,11793985205,7754610877001841,...,рак правой молочной железы,NaN,NaN,NaN,NaN,15.05.2020,6/211/20,15.05.2020,977.0,онко
4,2020-04-23,Лихачева Татьяна Михайловна,Женский,1979-11-10,Москвич,"г.Москва, Сиреневый бульвар, д.56, кв.112",Паспорт,NaN,06-219-793 27,1848 0208 8900 0065,...,рая яичников,NaN,NaN,NaN,NaN,23.04.2020,6/201/20,02.05.2020,59.0,онко
5,2023-03-03,Комарова Анастасия Анатольевна,Женский,1978-01-17,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",Паспорт,4521655829,032-653-064 26,7758 1208 8200 2822,...,ЗНО молочной железы,"Слабость, отеки ног, отсутствие аппетита, б/с",NaN,Почта,родственники,03.03.2023,6/101/23,01.03.2023,2.0,онко
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108004,2029-04-17,Гаврилова Наталья Евгеньевна,Женский,1950-06-13,Москвич,ул Ботаническая д 33 к 8 кв 103 п 1 эт 13 код 103,NaN,NaN,137-252-531 48,7700 0071 9313 065,...,Хорея Гентингтона,NaN,NaN,NaN,NaN,NaN,/1401/19,17.04.2019,-3394.0,nevrol
108005,2099-08-07,Муллаянов Рафаиль Мухлисуллович,Мужской,1954-11-13,Москвич,"Москва г, вн.тер.г. муниципальный округ Орехов...",Паспорт,4503/271725,019-635-971 83,7700 0080 6363 1154,...,NaN,Одышка. Слабость.,NaN,Телеграм,NaN,08.09.2022,5/1080/22,07.09.2022,-28053.0,teraph
108006,2202-04-12,Еникова Феня Мартковна,Женский,1925-11-18,Гражданин РФ/иногородний,Крым Феодосия Украинская 11,NaN,4514652788,18538624608,7768 4708 8100 0873,...,Деменция,NaN,NaN,NaN,NaN,NaN,1/339/21,12.04.2021,-66100.0,teraph
108007,2221-05-02,Тимофеева Татьяна Константиновна,Женский,1936-06-20,Москвич,Москва Западный Фили-Давыдково Большая Филёвск...,NaN,NaN,137-242-792 62,7773360879001871,...,"рак языка, губы",NaN,NaN,NaN,NaN,02.05.2021,12/349/21,02.05.2021,-73033.0,онко


In [141]:
# **************
df_result_star.groupby(['Тип_диагноза']).aggregate({'Тип_диагноза':'count'})
df_result_star.loc[df_result_star['Текущий статус пациента'] == 'Патронаж']

,Тип_диагноза
Тип_диагноза,
infek,19
nevrol,6730
not,1816
proch,686
psih,108
teraph,21119
vich,3
vrojden,95
онко,67326


In [147]:
# **************
df_result_star23.groupby(['Тип_диагноза']).aggregate({'Тип_диагноза':'count'})

,Тип_диагноза
Тип_диагноза,
nevrol,7364
not,1500
proch,228
psih,135
teraph,19999
vich,5
vrojden,96
онко,68159


In [171]:
df_result.loc[df_result['Диагноз'] == 'j69.23']

,Дата_постанов,Пациент,Пол,ДР,Иногород,Адрес,Документ удостоверяющий личность,Серия/№,СНИЛС,Полис ОМС,...,Основной диагноз на момент обращения,Комментарий,Участник СВО,Канал поступления информации,Кто направил,Дата получения ИДС на передачу данных,№ амбулаторной карты,"Дата включения в реестр пациентов, нуждающихся в ПМП",diff_days,Тип_диагноза
10686,NaT,Рыжова Марина Валерьевна,Женский,1982-11-03,Москвич,"129327, Москва г, муниципальный округ Бабушкин...",Паспорт,4514/839050,132-476-329 51,7748 7108 9600 0615,...,: C50.4 Злокачественное новообразование верхне...,26.10 недозвон\n\n28.10.2023 - звонок сестры. ...,нет,Почта,родственники,NaN,4/916/23,25.10.2023,NaN,teraph
11059,NaT,Сенчурова Анна Сергеевна,Женский,1952-11-05,Москвич,"127562, Москва г, муниципальный округ Отрадное...",Паспорт,4520/963414,125-177-571 58,7748 7408 9400 0796,...,ЗНО шейки матки,информация поступила по СМП (пассив) данные па...,NaN,Звонок,NaN,NaN,4/665/23,09.08.2023,NaN,teraph
70905,2021-06-30,Макеева Нина Григорьевна,Женский,1937-06-26,Москвич,"г.Москва, Бутырская, дом 21,кв21,под1,эт5, код...",Паспорт,4502511114,130-702-263 03,7700 0062 8826 0637,...,рак молочной железы,Дочь просила помочь с обезболиванием при обращ...,NaN,Почта,родственники,NaN,0/779/21,15.07.2022,561.0,teraph
85890,2022-08-09,Лашкевич Игорь Владимирович,Мужской,1966-10-14,Москвич,"125445, Москва г, муниципальный округ Ховрино,...",Паспорт,4511 094894,137-733-407 69,7700 0021 4364 1066,...,Инфекционный стафилококковый эндокардит,"ХБС, пролежень пятки, тяжесть в груди, агресси...",NaN,Самообращение,NaN,09.08.2022,0/712/22,09.08.2022,NaN,teraph
87104,2022-09-06,Кругликов Борис Петрович,Мужской,1938-06-21,Москвич,"Москва г, вн.тер.г. муниципальный округ Аэропо...",Паспорт,4504/566497,001-437-970 14,7700 0011 1471 0638,...,рак левой почки,"хбс, одышка, слабость",NaN,Самообращение,МГОБ № 62,09.09.2022,0/813/22,06.09.2022,211.0,teraph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107671,2023-10-31,Шабалин Владимир Назарович,Мужской,1942-01-21,Москвич,"127349, Москва г, муниципальный округ Бибирево...",Паспорт,4502633558,021-933-204 13,7778 7508 2800 0153,...,C20 - Злокачественное новообразование прямой к...,"Жалобы на боли в животе и г/боли, отказывается...",нет,Почта,родственники,21.10.2023,2/1479/23,16.10.2023,NaN,teraph
107714,2023-11-01,Селиванова Маргарита Викторовна,Женский,1951-05-13,Москвич,"125008, Москва г, муниципальный округ Коптево,...",NaN,NaN,001-515-985 09,7754 8408 8600 1017,...,С56 ЗНО яичника,"головокружение, недержание мочи, слабость, отс...",нет,Почта,ГП №6 ДЗМ,01.11.2023,2/1487/23,25.10.2023,NaN,teraph
107763,2023-11-02,Потапова Галина Викторовна,Женский,1963-02-04,Москвич,"123154, Москва г, муниципальный округ Хорошево...",NaN,4509 429401,001-400-196 58,7757 6308 9500 2768,...,рак головки поджелудочной железы мтс в печени,29.10.2023 пациентка 30.10.2023 госпитализируе...,нет,Почта,родственники,NaN,15/918/23,27.10.2023,NaN,teraph
107776,2023-11-02,Соселия Владимир Парменович,Мужской,1946-10-20,Москвич,"123458, Москва г, муниципальный округ Строгино...",Паспорт,4507/368336,065-047-479 65,7700 0050 8570 1046,...,C34.1 - Злокачественное новообразование верхне...,"Жалоба на слабость, боли в позвоночнике, на ру...",нет,Почта,родственники,02.11.2023,7//774/23,30.10.2023,NaN,teraph


In [115]:
df_result.to_excel('df_result_0311.xlsx')

In [436]:
df_result.columns.tolist()

['Дата_постанов',
 'Пациент',
 'Пол',
 'ДР',
 'Иногород',
 'Адрес',
 'Документ удостоверяющий личность',
 'Серия/№',
 'СНИЛС',
 'Полис ОМС',
 'Диагноз',
 'Дата_признания',
 'Тип документа о признании паллиативным',
 'Медицинская организация, в которой признали пациента нуждающимся в ПМП',
 'Наличие разрешения ДЗМ на оказание помощи',
 'Дата разрешения ДЗМ',
 'Номер разрешения ДЗМ',
 'Текущий статус пациента',
 'Дата_прекр',
 'Основания прекращения оказания пациенту ПМП',
 'Филиал',
 'Наблюдение в РЦ',
 'БК',
 'Основной диагноз на момент обращения',
 'Комментарий',
 'Участник СВО',
 'Канал поступления информации',
 'Кто направил',
 'Дата получения ИДС на передачу данных',
 'diff_days',
 'Тип_диагноза']

In [116]:
df_result[[
'Дата_постанов',
 'Пациент',
 'Пол',
 'ДР',
 'Иногород',
 'Адрес',
 'Документ удостоверяющий личность',
 'Серия/№',
 'СНИЛС',
 'Полис ОМС',
 'Диагноз',
 'Дата_признания',
 'Тип документа о признании паллиативным',
 'Медицинская организация, в которой признали пациента нуждающимся в ПМП',
 'Наличие разрешения ДЗМ на оказание помощи',
 'Дата разрешения ДЗМ',
 'Номер разрешения ДЗМ',
 'Текущий статус пациента',
 'Дата_прекр',
 'Основания прекращения оказания пациенту ПМП',
 'Филиал',
]]

,Дата_постанов,Пациент,Пол,ДР,Иногород,Адрес,Документ удостоверяющий личность,Серия/№,СНИЛС,Полис ОМС,...,Дата_признания,Тип документа о признании паллиативным,"Медицинская организация, в которой признали пациента нуждающимся в ПМП",Наличие разрешения ДЗМ на оказание помощи,Дата разрешения ДЗМ,Номер разрешения ДЗМ,Текущий статус пациента,Дата_прекр,Основания прекращения оказания пациенту ПМП,Филиал
0,2023-02-02,Зинатшина Анастасия Владимировна,Женский,1986-09-22,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",Паспорт,4519056415,201-154-322 87,7750 3108 7700 4903,...,2023-01-31,Заключение врачебного консилиума,МГОБ № 62,Нет,NaN,NaN,Умер,2023-02-02,NaN,Зеленоград ОВПП
1,2023-06-02,Олейникова Александра Павловна,Женский,1985-03-04,Москвич,"109341, Москва г, муниципальный округ Марьино,...",Паспорт,4520/883847,143-810-402 28,7756 4108 9500 0646,...,2023-05-17,Медицинское заключение для онкологических паци...,ГБУЗ «МГОБ № 62 ДЗМ» (ЦАОП 2 (ЗелАО)),Нет,NaN,NaN,Умер,2023-06-28,NaN,Зеленоград ОВПП
3,2020-05-15,Волченкова Юлия Викторовна,Женский,1983-05-22,Москвич,"Москва г, Зеленоград г, корпус 810, квартира 110",Паспорт,4614 693375,11793985205,7754610877001841,...,2022-03-09,Медицинское заключение для онкологических паци...,ГБУЗ «МГОБ № 62 ДЗМ»,Нет,NaN,NaN,Прекращено оказание ПМП,2023-01-17,Отказ пациента от получения ПМП,Зеленоград ОВПП
4,2020-04-23,Лихачева Татьяна Михайловна,Женский,1979-11-10,Москвич,"г.Москва, Сиреневый бульвар, д.56, кв.112",Паспорт,NaN,06-219-793 27,1848 0208 8900 0065,...,NaT,NaN,-,Нет,NaN,NaN,Умер,2020-06-21,NaN,Зеленоград ОВПП
5,2023-03-03,Комарова Анастасия Анатольевна,Женский,1978-01-17,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",Паспорт,4521655829,032-653-064 26,7758 1208 8200 2822,...,2023-03-01,Заключение врачебного консилиума,ГБУЗ «МГОБ № 62 ДЗМ»,Нет,NaN,NaN,Умер,2023-03-05,NaN,Зеленоград ОВПП
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107776,2033-02-08,Бородина Валентина Вениаминовна,Женский,1945-01-30,Москвич,"Москва г, вн.тер.г. муниципальный округ Марьин...",Паспорт,4523 259825,026-779-576 04,7778 4508 6900 0770,...,2023-01-27,Медицинское заключение для онкологических паци...,Филиал ГБУЗ «МКНЦ им. А.С. Логинова ДЗМ» ЦАОП,Нет,NaN,NaN,Патронаж,NaT,NaN,Преображенское ОВПП
107777,2099-08-07,Муллаянов Рафаиль Мухлисуллович,Мужской,1954-11-13,Москвич,"Москва г, вн.тер.г. муниципальный округ Орехов...",Паспорт,4503/271725,019-635-971 83,7700 0080 6363 1154,...,2022-09-07,Заключение врачебной комиссии,ГБУЗ ГП 214 ДЗМ,Нет,NaN,NaN,Умер,2022-10-17,NaN,Царицыно ОВПП
107778,2202-04-12,Еникова Феня Мартковна,Женский,1925-11-18,Гражданин РФ/иногородний,Крым Феодосия Украинская 11,NaN,4514652788,18538624608,7768 4708 8100 0873,...,NaT,NaN,-,Нет,NaN,NaN,Умер,2021-04-20,NaN,ПМХ ОВПП
107779,2221-05-02,Тимофеева Татьяна Константиновна,Женский,1936-06-20,Москвич,Москва Западный Фили-Давыдково Большая Филёвск...,NaN,NaN,137-242-792 62,7773360879001871,...,2021-05-14,Заключение врачебной комиссии,КДЦ 4,Нет,NaN,NaN,Умер,2021-05-17,NaN,Хорошево ОВПП


In [134]:
df_result.loc[df_result['Тип_диагноза'] == 'онко']

,Дата_постанов,Пациент,Пол,ДР,Иногород,Адрес,Документ удостоверяющий личность,Серия/№,СНИЛС,Полис ОМС,...,БК,Основной диагноз на момент обращения,Комментарий,Участник СВО,Канал поступления информации,Кто направил,Дата получения ИДС на передачу данных,diff_days,Тип_диагноза,Признак_неправильной_МО
0,2023-02-02,Зинатшина Анастасия Владимировна,Женский,1986-09-22,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",Паспорт,4519056415,201-154-322 87,7750 3108 7700 4903,...,Нет,Рак желулка,"Со слов мужа Не ест ( тошнота),большая потеря ...",NaN,Почта,родственники,02.02.2023,0.0,онко,NaN
1,2023-06-02,Олейникова Александра Павловна,Женский,1985-03-04,Москвич,"109341, Москва г, муниципальный округ Марьино,...",Паспорт,4520/883847,143-810-402 28,7756 4108 9500 0646,...,Нет,C18.0 - Злокачественное новообразование слепой...,"боли, НПВС без эффекта, поднимается иногда тем...",NaN,Почта,родственники,02.06.2023,26.0,онко,NaN
3,2020-05-15,Волченкова Юлия Викторовна,Женский,1983-05-22,Москвич,"Москва г, Зеленоград г, корпус 810, квартира 110",Паспорт,4614 693375,11793985205,7754610877001841,...,Нет,рак правой молочной железы,NaN,NaN,NaN,NaN,15.05.2020,977.0,онко,NaN
4,2020-04-23,Лихачева Татьяна Михайловна,Женский,1979-11-10,Москвич,"г.Москва, Сиреневый бульвар, д.56, кв.112",Паспорт,NaN,06-219-793 27,1848 0208 8900 0065,...,Нет,рая яичников,NaN,NaN,NaN,NaN,23.04.2020,59.0,онко,Некорректная_МО
5,2023-03-03,Комарова Анастасия Анатольевна,Женский,1978-01-17,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",Паспорт,4521655829,032-653-064 26,7758 1208 8200 2822,...,Нет,ЗНО молочной железы,"Слабость, отеки ног, отсутствие аппетита, б/с",NaN,Почта,родственники,03.03.2023,2.0,онко,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107773,2023-12-17,Дюжев Владимир Александрович,Мужской,1955-06-06,Москвич,"Москва г, вн.тер.г. муниципальный округ Голови...",Паспорт,4515/077195,019-494-226 71,7753 4408 4300 0607,...,Нет,"С34.1 зно верхней доли легкого, бронхов",Сильные боли в ногах до колен. Необходим подбо...,NaN,Телеграм,родственники,17.12.2022,NaN,онко,NaN
107774,2023-12-25,Бабичева Людмила Ивановна,Женский,1947-01-10,Москвич,"Москва г, вн.тер.г. муниципальный округ Донско...",Паспорт,4505-142845,001-583-592 23,7700 0021 2610 0147,...,Нет,С18.7 рак сигмовидной кишки,Подтверждать визиты с пациенткой \nБолят кишки...,NaN,Почта,ГП № 22 ф.3,NaN,NaN,онко,NaN
107776,2033-02-08,Бородина Валентина Вениаминовна,Женский,1945-01-30,Москвич,"Москва г, вн.тер.г. муниципальный округ Марьин...",Паспорт,4523 259825,026-779-576 04,7778 4508 6900 0770,...,Нет,С 73 ЗНО щитовидной железы,"ЗНО щитовидной железы, метастазы печени\nнет ф...",NaN,Почта,ГБУЗ ОД 5 ДЗМ,08.02.2033,NaN,онко,NaN
107779,2221-05-02,Тимофеева Татьяна Константиновна,Женский,1936-06-20,Москвич,Москва Западный Фили-Давыдково Большая Филёвск...,NaN,NaN,137-242-792 62,7773360879001871,...,Нет,"рак языка, губы",NaN,NaN,NaN,NaN,02.05.2021,-73033.0,онко,NaN


In [118]:
df_result.loc[(df_result['Текущий статус пациента'] == 'Патронаж' ) & (df_result['Тип_диагноза'] != 'онко') ]

,Дата_постанов,Пациент,Пол,ДР,Иногород,Адрес,Документ удостоверяющий личность,Серия/№,СНИЛС,Полис ОМС,...,БК,Основной диагноз на момент обращения,Комментарий,Участник СВО,Канал поступления информации,Кто направил,Дата получения ИДС на передачу данных,diff_days,Тип_диагноза,Признак_неправильной_МО
14,2022-10-26,Рябенков Андрей Владимирович,Мужской,1973-10-17,Москвич,"Москва г, Зеленоград г, корпус 309, квартира 3...",Паспорт,4518876899,126-426-618 52,7700 0050 2967 1073,...,Нет,ОНМК,"Трофические язвы. В пределах кровати активен, ...",NaN,Почта,NaN,26.10.2022,NaN,teraph,NaN
20,2023-04-09,Жук Людмила Александровна,Женский,1968-09-18,Москвич,"Москва г, вн.тер.г. муниципальный округ Южное ...",Паспорт,9213/597061,054-163-821 44,1650 1308 8100 0091,...,Нет,эозинофильный гранулематоз,"Одышка, слабость, приступы удушья\nЖдем гр. до...",NaN,Почта,ГП №219 ф.2,09.04.2023,NaN,teraph,NaN
32,2023-02-24,Жуков Владимир Владимирович,Мужской,1965-05-24,Москвич,"Москва г, Зеленоград г, корпус 1436, квартира 165",Паспорт,4510/555862,013-039-260 98,7700 0020 3174 0565,...,Нет,ЦВБ. ХИГМ 3ст,NaN,NaN,Самообращение,ГБУЗ ГКБ им. М.П. Кончаловского ДЗМ,24.02.2023,NaN,teraph,NaN
37,2023-05-18,Морозова Наталья Анатольевна,Женский,1963-08-27,Москвич,"Москва г, вн.тер.г. муниципальный округ Нагати...",Паспорт,4509 663052,005-739-463 54,7700 0061 0527 0863,...,Нет,последствия инфаркта головного мозга,"ФНКЦ РР (д.Лыткино), корпус А, отд.реанимации ...",NaN,Почта,ФНКЦ РР Лыткино,18.05.2023,NaN,teraph,NaN
55,2023-06-15,Чижиковская Людмила Владиславовна,Женский,1959-10-01,Москвич,"124575, Москва г, вн.тер.г. муниципальный окру...",Паспорт,4508102751,079-072-357 87,7749 0408 9800 2719,...,Нет,ЦВБ. ХИГМ.,"нет гр. д-ов, сестра вышлет на почту\nСлабость...",NaN,Почта,ГБУЗ ГКБ им. М.П. Кончаловского ДЗМ ПО№ 5,15.06.2023,NaN,teraph,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107757,2023-11-02,Черненко Леонид Николаевич,Мужской,1934-09-18,Москвич,"Москва г, Большая Очаковская ул, Дом 12, Корпу...",Паспорт,4523 195752,218-034-644 42,7770 5608 3100 0639,...,Нет,С44.3 Другие злокачественные новообразования кожи,Пациент из реестра. Есть документ о нуждаемост...,нет,Почта,ГБУЗ ГКОБ №1 ДЗМ ЦАОП,02.11.2023,NaN,not,NaN
107759,2023-11-02,Щеглова Ангелина Кузьминична,Женский,1946-02-19,Москвич,"111538, Москва г, вн.тер.г. муниципальный окру...",Паспорт,4509/857159,123-546-696 61,7777 3508 8000 1248,...,Нет,C50.1 – Злокачественное новообразование центра...,30.10.2023 - автоответчик - оставлено голосово...,нет,Почта,ГП №66,02.11.2023,NaN,not,NaN
107760,2023-11-02,Яшин Михаил Юрьевич,Мужской,1962-01-12,Москвич,"109457, Москва г, муниципальный округ Кузьминк...",Паспорт,4509/249663,001-552-622 94,7758 7308 3700 1347,...,Нет,ЦВБ. Инфаркт головного мозга,"Жалобы: на всю спину красное пятно, шелушится,...",нет,Почта,"ЮВАО, ГП №9",02.11.2023,NaN,teraph,NaN
107762,2023-11-03,Крупчатников Александр Валивиевич,Женский,1970-08-22,Москвич,"109388, Москва г, муниципальный округ Печатник...",Паспорт,4518 993707,058-238-631 79,7700 0001 0472 0870,...,Нет,ИБС,со слов пациента открытые трофические язвы на ...,NaN,Звонок,пациент,03.11.2023,NaN,teraph,NaN


In [119]:
df_result['Текущий статус пациента'].unique()

array(['Умер', 'Прекращено оказание ПМП', 'Патронаж', 'Стационар ЦПП',
       'Госпитализация в ОПМП', nan], dtype=object)

In [132]:
df_result.loc[(df_result['Текущий статус пациента'] == 'Патронаж' ) & (df_result['Тип_диагноза'] != 'онко') ].shape

(8026, 32)

In [128]:
df_result.loc[(df_result['Текущий статус пациента'] == 'Стационар ЦПП' ) & (df_result['Тип_диагноза'] == 'онко') ].shape

(613, 32)

In [127]:
df_result.loc[(df_result['Текущий статус пациента'] == 'Госпитализация в ОПМП' ) & (df_result['Тип_диагноза'] == 'онко') ].shape

(5, 32)

In [130]:
5137 + 220+5

5362

In [133]:
# df_result['Филиал'].unique()

array(['Зеленоград ОВПП', 'Дегунино ОВПП', 'Савеловский ОВПРП',
       'Хорошево ОВПП', 'Куркино ОВПП', 'Бутово ОВПП', 'ПМДХ ОВПП',
       'Солнцево ОВПП', 'ПМХ ОВПП', 'Савеловский ОВПП', 'Люблино ОВПП',
       'Вороново ОВПП', 'Ростокино ОВПП', 'Царицыно ОВПП',
       'Центр ОВПРП ул. Двинцев, 6, стр. 2, Москва, Россия',
       'Центр ОВПП(ул. Двинцев, 6, стр. 2, Москва, Россия)',
       'Не использовать - дубли', 'Перово ОВПП', 'Черемушки ОВПП',
       'Некрасовка ОВПП', 'Преображенское ОВПП', nan, 'Хоспис "Дегунино"',
       'Хоспис "Савёловский"', 'Хоспис "Ростокино"', 'Хоспис "Куркино"',
       'Хоспис "Бутово"'], dtype=object)